In [13]:
import pandas as pd
import numpy as np
from scipy.special import rel_entr, entr

In [14]:
# List of available dataset
datasets = ["DTD","EuroSAT","GTSRB","MNIST","SVHN","Caltech256"]

In [15]:
def calc_kl(P, Q):
    return np.sum(rel_entr(P, Q), axis=1)

# Calculate Entropy
def calc_entropy(P):
    return np.sum(entr(P), axis=1)

# Calculate Cross-Entropy
def calc_cross_ent(P, Q):
    # return np.sum(entr(P,Q), axis=1)
    return -np.sum(P * np.log(Q), axis=1)


In [16]:
for ds in datasets:
    df18 = pd.read_csv(f"../Pred/{ds}_Resnet18.csv")
    df50 = pd.read_csv(f"../Pred/{ds}_Resnet50.csv")
    df101 = pd.read_csv(f"../Pred/{ds}_Resnet101.csv")
    label = pd.read_csv(f"../Pred/target_{ds}.csv")
    prediction = pd.DataFrame({
    "pred18": df18.idxmax(axis=1),
    "pred50": df50.idxmax(axis=1),
    "pred101": df101.idxmax(axis=1),
    "target": label['target']
    })
    prediction["pred18"] = prediction["pred18"].str.extract('(\d+)').astype(int)
    prediction["pred50"] = prediction["pred50"].str.extract('(\d+)').astype(int)
    prediction["pred101"] = prediction["pred101"].str.extract('(\d+)').astype(int)
    prediction.to_csv(f"prediction_{ds}.csv")
    unc_pred = pd.DataFrame()
    unc_pred["kl_18_101"] = calc_kl(df18,df101)
    unc_pred["kl_101_18"] = calc_kl(df101,df18)
    unc_pred["ent_18"] = calc_entropy(df18)
    unc_pred["ent_101"] = calc_entropy(df101)
    unc_pred["ce_18_101"] = calc_cross_ent(df18,df101)
    unc_pred["ce_101_18"] = calc_cross_ent(df101,df18)
    unc_pred.to_csv(f"uncertainty_{ds}.csv")